In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
import pickle
import numpy as np
import os
import pandas as pd

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving sherlockholmes.txt to sherlockholmes (1).txt


In [ ]:
# Building the Dataset from .txt
file = open("sherlockholmes.txt", "r", encoding = "utf8")
lines = []
for i in file:
    lines.append(i)

data = []
for i in lines:
#   data = ' '. join(lines) 

  i = i.replace('\n', '').replace('\r', '').replace('\ufeff', '').replace('“','').replace('”','').replace('I.','').replace('_','').replace('.','')
  i = i.split()
  data.append(' '.join(i))

data_final = []
for i in range(0,len(data)-1):
  if data[i] != '':
    data_final.append(data[i])

del data_final[0:29]
del data_final[-309:-1]
sentz = pd.DataFrame(data_final)
sentz.columns = ['sentenz']
print(sentz['sentenz'])
#sentence count 
len(data)

0                                    A SCANDAL IN BOHEMIA
1       To Sherlock Holmes she is always the woman I h...
2       mention her under any other name In his eyes s...
3       predominates the whole of her sex It was not t...
4       akin to love for Irene Adler All emotions, and...
                              ...                        
9289    Southampton the day after their flight, and he...
9290    government appointment in the island of Maurit...
9291    Hunter, my friend Holmes, rather to my disappo...
9292    further interest in her when once she had ceas...
9293    subscribe to our email newsletter to hear abou...
Name: sentenz, Length: 9294, dtype: object


12310

In [ ]:
!python -m nltk.downloader punkt 
!python -m nltk.downloader stopwords 
!python -m nltk.downloader wordnet
!python -m nltk.downloader omw-1.4 

/usr/lib/python3.7/runpy.py:125: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
/usr/lib/python3.7/runpy.py:125: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
/usr/lib/python3.7/runpy.py:125: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package wordnet to /root/nltk_data...
/usr/lib/pyth

In [ ]:
#Tokenizing and Vectorizing 
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.lancaster import LancasterStemmer
lancaster_stemmer = LancasterStemmer()
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
from tqdm import tqdm

def tokenize(tweet):
    lower_tweet = tweet.lower()
    return word_tokenize(lower_tweet)

def remove_stop_words(tokens):
    clean_tokens = []
    stopWords = set(stopwords.words('english'))
    for token in tokens:
        if token not in stopWords:
            if token.replace(' ', '') != '':
                if len(token) > 1:
                    clean_tokens.append(token)
    return clean_tokens

def stem_and_lem(tokens):
    clean_tokens = []
    for token in tokens:
        token = wordnet_lemmatizer.lemmatize(token)
        token = lancaster_stemmer.stem(token)
        if len(token) > 1:
            clean_tokens.append(token)
    return clean_tokens

tqdm.pandas(desc="Tokenizing Data...")
sentz['sentenz'] = sentz['sentenz'].progress_apply(tokenize)

# tqdm.pandas(desc="Cleaning Data Phase II...")
# sentz['sentenz'] = sentz['sentenz'].progress_apply(remove_stop_words)

# tqdm.pandas(desc="Stemming And Lemmatizing")
# sentz['sentenz'] = sentz['sentenz'].progress_apply(stem_and_lem)

sentz.to_csv("new_text_vectors.csv")
text_vector = sentz['sentenz'].tolist()

final_tokens = []
for i in text_vector:
  if len(i) != 0:
    final_tokens.append(i)
print(final_tokens)

Tokenizing Data...: 100%|██████████| 9602/9602 [00:01<00:00, 7780.92it/s]


[['a', 'scandal', 'in', 'bohemia'], ['to', 'sherlock', 'holmes', 'she', 'is', 'always', 'the', 'woman', 'i', 'have', 'seldom', 'heard', 'him'], ['mention', 'her', 'under', 'any', 'other', 'name', 'in', 'his', 'eyes', 'she', 'eclipses', 'and'], ['predominates', 'the', 'whole', 'of', 'her', 'sex', 'it', 'was', 'not', 'that', 'he', 'felt', 'any', 'emotion'], ['akin', 'to', 'love', 'for', 'irene', 'adler', 'all', 'emotions', ',', 'and', 'that', 'one', 'particularly', ','], ['were', 'abhorrent', 'to', 'his', 'cold', ',', 'precise', 'but', 'admirably', 'balanced', 'mind', 'he'], ['was', ',', 'i', 'take', 'it', ',', 'the', 'most', 'perfect', 'reasoning', 'and', 'observing', 'machine', 'that'], ['the', 'world', 'has', 'seen', ',', 'but', 'as', 'a', 'lover', 'he', 'would', 'have', 'placed', 'himself', 'in', 'a'], ['false', 'position', 'he', 'never', 'spoke', 'of', 'the', 'softer', 'passions', ',', 'save', 'with', 'a', 'gibe'], ['and', 'a', 'sneer', 'they', 'were', 'admirable', 'things', 'for', 